In [ ]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout



In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/complaints.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
col = ['Product', 'Consumer complaint narrative']
df = df[col]

In [ ]:
df

,Product,Consumer complaint narrative
0,"Credit reporting, credit repair services, or o...",NaN
1,"Credit reporting, credit repair services, or o...",NaN
2,Vehicle loan or lease,I contacted Ally on Friday XX/XX/XXXX after fa...
3,"Credit reporting, credit repair services, or o...",NaN
4,"Credit reporting, credit repair services, or o...",NaN
...,...,...
566331,"Credit reporting, credit repair services, or o...",NaN
566332,"Credit reporting, credit repair services, or o...",In violation of FCRA 605b and Section 611 ( a ...
566333,"Credit reporting, credit repair services, or o...",NaN
566334,"Credit reporting, credit repair services, or o...",NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566336 entries, 0 to 566335
Data columns (total 2 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Product                       566336 non-null  object
 1   Consumer complaint narrative  258747 non-null  object
dtypes: object(2)
memory usage: 8.6+ MB


In [ ]:
df=df.dropna()

In [ ]:
df.reset_index()

,index,Product,Consumer complaint narrative
0,2,Vehicle loan or lease,I contacted Ally on Friday XX/XX/XXXX after fa...
1,5,"Money transfer, virtual currency, or money ser...",On XX/XX/XXXX I was logged out of my coinbase....
2,8,"Credit reporting, credit repair services, or o...",Hello This complaint is against the three cred...
3,9,"Credit reporting, credit repair services, or o...",I am a victim of Identity Theft & currently ha...
4,10,Student loan,My loan Federal loan was transferred from Sall...
...,...,...,...
258742,566323,"Credit reporting, credit repair services, or o...",In XXXX of 2020 I parked my XXXX vehicle and i...
258743,566324,"Credit reporting, credit repair services, or o...",To : Whom It May concern : I am writing to dis...
258744,566329,Debt collection,TO WHOM IT MAY CONCERN XXXX XXXX XXXX Despite ...
258745,566332,"Credit reporting, credit repair services, or o...",In violation of FCRA 605b and Section 611 ( a ...


In [ ]:
df.reset_index(drop=True)

,Product,Consumer complaint narrative
0,Vehicle loan or lease,I contacted Ally on Friday XX/XX/XXXX after fa...
1,"Money transfer, virtual currency, or money ser...",On XX/XX/XXXX I was logged out of my coinbase....
2,"Credit reporting, credit repair services, or o...",Hello This complaint is against the three cred...
3,"Credit reporting, credit repair services, or o...",I am a victim of Identity Theft & currently ha...
4,Student loan,My loan Federal loan was transferred from Sall...
...,...,...
258742,"Credit reporting, credit repair services, or o...",In XXXX of 2020 I parked my XXXX vehicle and i...
258743,"Credit reporting, credit repair services, or o...",To : Whom It May concern : I am writing to dis...
258744,Debt collection,TO WHOM IT MAY CONCERN XXXX XXXX XXXX Despite ...
258745,"Credit reporting, credit repair services, or o...",In violation of FCRA 605b and Section 611 ( a ...


In [ ]:
df.Product.value_counts()

Credit reporting, credit repair services, or other personal consumer reports    132639
Debt collection                                                                  44086
Credit card or prepaid card                                                      24793
Mortgage                                                                         19470
Checking or savings account                                                      15089
Money transfer, virtual currency, or money service                                7118
Vehicle loan or lease                                                             5787
Student loan                                                                      5302
Payday loan, title loan, or personal loan                                         3879
Credit reporting                                                                   163
Credit card                                                                        117
Bank account or service                    

In [ ]:
df.Product.unique()

array(['Vehicle loan or lease',
       'Money transfer, virtual currency, or money service',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Student loan', 'Credit card or prepaid card', 'Mortgage',
       'Payday loan, title loan, or personal loan', 'Debt collection',
       'Checking or savings account', 'Credit reporting', 'Consumer Loan',
       'Credit card', 'Bank account or service', 'Money transfers',
       'Prepaid card', 'Payday loan', 'Other financial service'],
      dtype=object)

In [ ]:
df['label']=df['Product'].factorize()[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df = df.reset_index(drop=True)

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [ ]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [ ]:
df['clean_txt']=df['Consumer complaint narrative'].apply(clean_text)

In [ ]:
df['clean_txt'] = df['clean_txt'].str.replace('\d+', '')

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['clean_txt'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 96384 unique tokens.


In [ ]:
X = tokenizer.texts_to_sequences(df['Consumer complaint narrative'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (258747, 250)


In [ ]:
Y = pd.get_dummies(df['Product']).values

In [ ]:
Y

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(232872, 250) (232872, 17)
(25875, 250) (25875, 17)


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(17, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 17)                1717      
                                                                 
Total params: 5,082,117
Trainable params: 5,082,117
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
3275/3275 [==============================] - 3131s 955ms/step - loss: 1.0267 - accuracy: 0.6584 - val_loss: 0.7604 - val_accuracy: 0.7397
Epoch 2/5
 955/3275 [=======>......................] - ETA: 36:41 - loss: 0.6947 - accuracy: 0.7725

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
